
# API TWITTER 
Objective:
This api has the objective to collect data from stock market inside a tweet api.
The focus of the API is to process info for IBOVESPA market.

<h1> 1. Collect and process data inside a mongoDB database </h1>

# 1.1 Import packages

In [1]:
#Api packages
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
#packages for data manipulation
from datetime import datetime
import json
import pickle
import pandas as pd
#packages for storage
from pymongo import MongoClient

In [37]:
#extract the api credentials for pickles files
api_key_df = pd.read_pickle(r'crendentials_key.pkl')
api_token_df = pd.read_pickle(r'crendentials_token.pkl')

In [2]:
access_token = '725464864861331456-Dj9KxUbWuTjjES6fU11YJoJhqfWSoHb'
api_key = 'V8bym5SOBxQziEJUiyv63725L'
api_key_secret = 'CatdyRop1VLZQ4uCHKwZh8bMI6B3w46qtsOlREGrbpmtATOuP7'
access_token='725464864861331456-Dj9KxUbWuTjjES6fU11YJoJhqfWSoHb'
access_token_secret= 'ElRX10ABnyU8jabnMfZK5utMkTOIaom9oV2KRktDNSkQM'

In [3]:
#Connect with the Twitter API
auth = OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
auth

In [6]:
#Connect with the mongodb
conn = MongoClient('localhost',27017)
print(conn)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)


In [8]:
#create a database
db = conn.twitterdb

In [9]:
#input colls inside the database
col = db.tweets

# 2. Collect Data from Tweet API

In [16]:
#Create a Class for get the tweets data and storage in the mongodb database
class Twitter_Collector(StreamListener):
    def on_data(self, data):
        tweet = json.loads(data)
        created_at = tweet['created_at']
        id_str = tweet['id']
        tweet_text = tweet['text']
        obj =  {"created_at":created_at,"id_str":id_str,"text":tweet_text,}
        tweetind = col.insert_one(obj).inserted_id
        print(obj)
        return True

In [17]:
#create the mylistener object
ttcc = Twitter_Collector() 

In [23]:
#create the stream object
stream = Stream(auth, listener= ttcc)

In [24]:
#keywords
keywords = ['MXRF11','fundos imobiliários','fiis']

In [26]:
stream.filter(track=keywords)

{'created_at': 'Thu Mar 11 18:40:25 +0000 2021', 'id_str': 1370082198557229059, 'text': 'pandemia, que acarretou no crescimento do e-commerce (comercio eletrônico), trazendo novos investimentos e investid… https://t.co/tVbDC8QCb5', '_id': ObjectId('604a641e69d6bd9f44f95895')}
{'created_at': 'Thu Mar 11 18:40:26 +0000 2021', 'id_str': 1370082200855666690, 'text': '2020. \n\n➡️ Além das captações bem-sucedidas ao longo do ano, os fundos imobiliários deste setor encerraram o ano se… https://t.co/8csgTmKfiP', '_id': ObjectId('604a641f69d6bd9f44f95896')}
{'created_at': 'Thu Mar 11 18:41:31 +0000 2021', 'id_str': 1370082473292484618, 'text': '@RafaelPaschoare O mesmo racional se aplica a fdic para recebiveis, fip para equity e fia para ações. Estruturas ca… https://t.co/vmJvKwKbHB', '_id': ObjectId('604a646069d6bd9f44f95897')}
{'created_at': 'Thu Mar 11 18:42:03 +0000 2021', 'id_str': 1370082608235876356, 'text': '@RafaelPaschoare O mesmo racional se aplica a fdic para recebiveis, fip para e

KeyboardInterrupt: 

In [21]:
stream.disconnect()

In [22]:
dataset = [{"created_at": item['created_at'], "text": item["text"],}for item in col.find()]